In [1]:
%load_ext autoreload
%autoreload 2
!pwd

/home/ajasja/projects/truncator


In [2]:
import dask
from dask_jobqueue import SLURMCluster
cluster = SLURMCluster(cores=1,
                     memory="2GB",
                     queue='short',
                     walltime="3:00:00")
#cluster.scale(40)
cluster.adapt(minimum=2, maximum=50, interval='5s', wait_count=240, target_duration='180s')
print(cluster.scheduler_address); print(cluster.dashboard_link)
from dask.distributed import Client, progress
client = Client(cluster);
client.write_scheduler_file('scheduler.json');
client

tcp://172.16.131.63:29303
http://172.16.131.63:8787/status


Client Scheduler: tcp://172.16.131.63:29303 Dashboard: http://172.16.131.63:8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [4]:
!squeue -u ajasja

ARRAY_JO ARRAY PRIORI PARTITIO USER       START_TIME           ST TIME     MIN_ CPUS NAME                 NODELIST REASON              
6316838  N/A   1999   short    ajasja     2019-02-26T15:55:30  R  0:02     2G   1    dask-worker          dig59    None                
6316839  N/A   1999   short    ajasja     2019-02-26T15:55:30  R  0:02     2G   1    dask-worker          dig59    None                
6300249  N/A   1980   long     ajasja     2019-02-26T09:45:02  R  6:10:30  8G   2    jupyter-session      dig33    None                


In [11]:
!scancel 5038215   5038216  

In [13]:
!mv  out/R3/04_relooped 

In [14]:
def task_to_perform(astr):
    print("task in progress.")
    print(astr)
    print("task is over.!")
    return astr
res = []
for rs in range(10,40):
    res.append(client.submit(task_to_perform, rs))
    


In [15]:
res[0].result()

10

In [3]:
from dask.distributed import Client
client = Client(cluster)

In [8]:
client

In [15]:
cluster.close()

In [5]:
pdb_files = glob("testing_out/grouped/t3/*.pdb")
pdb_files

['testing_out/grouped/t3/ZCON_37_N3C1_3.pdb',
 'testing_out/grouped/t3/ZCON_131_1.pdb',
 'testing_out/grouped/t3/ZCON_37_N3C1_2.pdb',
 'testing_out/grouped/t3/ZCON_39_1.pdb',
 'testing_out/grouped/t3/ZCON_131_2.pdb']

In [6]:
%%time
if False:
    sf= []
    for pdb in pdb_files:
        sf.append(truncator.score_interface(pdb,
                        script_name="truncator/xml/04_FD_FR_and_scoring_benchmark_v3_digs_updated_schema_BUNS3_IA_select.xml",
                        out_dir="testing_out/grouped/t1", test_run=False))
    sf

CPU times: user 5 µs, sys: 1e+03 ns, total: 6 µs
Wall time: 10.5 µs


In [7]:
%%time
sf= []
for pdb in pdb_files:
    sf.append(client.submit(truncator.score_interface, pdb,
                    script_name="truncator/xml/04_FD_FR_and_scoring_benchmark_v3_digs_updated_schema_BUNS3_IA_select.xml",
                    out_dir="testing_out/grouped/t3", test_run=False))
sf

CPU times: user 1.92 ms, sys: 4.19 ms, total: 6.12 ms
Wall time: 5.01 ms


In [13]:
sf[3].result()

'testing_out/grouped/t2/ZCON_37_3+1__numH4__from-22.25__to17.45__grAB-CD_IA.sc'

In [9]:
score_files = sorted([r.result() for r in sf], reverse=True)

In [2]:
score_files = ['testing_out/grouped/t3/ZCON_39_1_IA.sc',
 'testing_out/grouped/t3/ZCON_37_N3C1_3_IA.sc',
 'testing_out/grouped/t3/ZCON_37_N3C1_2_IA.sc',
 'testing_out/grouped/t3/ZCON_131_2_IA.sc',
 'testing_out/grouped/t3/ZCON_131_1_IA.sc']

In [6]:

scores = truncator.read_score_files(score_files, skiprows=False, load_seq=False).set_index('decoy', drop=True)
with pd.option_context("display.max_rows", 1000, "display.max_colwidth", 80, "display.width", 50):
    display(scores.transpose())

decoy,ZCON_39_1,ZCON_37_N3C1_3,ZCON_37_N3C1_2,ZCON_131_2,ZCON_131_1
NetCharge_A,-8,-12,-6,-2,-4
NetCharge_B,-1,-2,-3,-9,-11
VirtualRootMover_root,159,92,120,94,122
average_res_energy_IA,-4.48534,-3.96393,-3.40572,-3.18815,-3.78778
cavity_volume,101.599,36.1283,42.977,0,0
complex_normalized,-4.04337,-3.57129,-3.03114,-2.92105,-3.42747
cross_interface_energy_IA,-0.045358,-0.0440467,-0.0398013,-0.0415372,-0.0430931
cross_interface_hbonds_IA,16,8,7,8,15
dG_cross,-102.409,-99.6324,-102.183,-94.6677,-135.099
dG_cross/dSASAx100,-4.5358,-4.40467,-3.98013,-4.15372,-4.30931


In [11]:
with pd.option_context("display.max_rows", 1000, "display.max_colwidth", 500, "display.width", 50):
    display(scores['pymol_interface_selection_IA'])

decoy
ZCON_39_1                                                                    select ZCON_39_1_interface, /ZCON_39_1//A/2+3+5+6+9+10+12+13+14+16+17+19+20+21+23+24+27+28+29+30+31+32+33+36+37+39+40+41+42+43+44+45+46+47+48+50+51+ + /ZCON_39_1//B/80+82+83+84+86+87+90+91+93+94+96+97+98+100+101+103+107+108+110+111+112+114+115+117+118+119+121+122+123+124+125+126+127+128+129+130+
ZCON_37_N3C1_3                                                                  select ZCON_37_N3C1_3_interface, /ZCON_37_N3C1_3//A/3+4+7+10+11+14+17+21+24+25+26+27+28+29+30+31+32+33+34+35+36+37+38+39+40+41+42+43+44+45+46+47+50+51+52+53+54+55+56+57+58+59+60+61+62+63+64+65+66+67+68+69+70+71+ + /ZCON_37_N3C1_3//B/72+73+74+75+76+77+78+79+80+81+82+83+84+85+86+87+88+89+90+91+
ZCON_37_N3C1_2    select ZCON_37_N3C1_2_interface, /ZCON_37_N3C1_2//A/3+4+7+10+14+21+24+28+30+31+32+33+34+35+36+39+40+42+43+44+45+46+47+48+49+50+51+52+53+54+55+56+57+58+60+63+64+65+67+68+69+70+71+72+73+74+75+76+77+78+79+80+81+82+83+85+86+88+89+90

In [4]:
def submit_pdb(pdb):
    import os
    os.chdir('/home/ajasja/projects/truncator')
    import truncator
    return truncator.score_interface(pdb,
                    script_name="truncator/xml/03_FD_FR_and_scoring_benchmark_v3_digs_updated_schema_BUNS3_IA_quick.xml",
                    out_dir="testing_out/grouped")

In [21]:
score

,SCORE:,total_score,NetCharge_A,NetCharge_B VirtualRootMover_root,cart_bonded_monomer cavity_volume,complex_normalized,dG_cross,dG_cross/dSASAx100,dG_separated,dG_separated/dSASAx100,...,Unnamed: 106,Unnamed: 107,Unnamed: 108,Unnamed: 109,Unnamed: 110,Unnamed: 111,Unnamed: 112,Unnamed: 113,Unnamed: 114,Unnamed: 115
0,SCORE:,-360.706,-3.0,-6.000 120.000,0.000 0.000,-3.031,-102.183,-3.98,-93.888,-3.657,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,SCORE:,-360.706,2.0,-6.000 -3.000,675.431 2664.716,1.340,0.000,0.00,-3.031,32.000,...,0.0,0.896,1.0,-48.399,0.0,3.0,1.0,1.0,0.0,ZCON_37_N3_C1_2


/software/rosetta/latest/bin/rosetta_scripts     -parser:protocol truncator/xml/04_FD_FR_and_scoring_benchmark_v3_digs_updated_schema_BUNS3_IA_select.xml -s testing_out/grouped/ZCON_37_N3_C1_2.pdb     -out:file:scorefile  testing_out/grouped/ZCON_37_N3_C1_2.sc     -out:file:o  testing_out/grouped/ZCON_37_N3_C1_2.pdb     -print_pymol_selection 0     -beta     -in:file:fullatom     -renumber_pdb 1     -overwrite     -out:no_nstruct_label     -holes:dalphaball truncator/scripts/DAlphaBall.gcc     > testing_out/grouped/ZCON_37_N3_C1_2.log
testing_out/grouped/ZCON_37_N3_C1_2.sc
Saving to: testing_out/grouped/ZCON_37_N3_C1_2_IA.sc
/software/rosetta/latest/bin/rosetta_scripts     -parser:protocol truncator/xml/04_FD_FR_and_scoring_benchmark_v3_digs_updated_schema_BUNS3_IA_select.xml -s testing_out/grouped/ZCON_37_3+1__numH4__from-22.25__to17.45__grAB-CD.pdb     -out:file:scorefile  testing_out/grouped/ZCON_37_3+1__numH4__from-22.25__to17.45__grAB-CD.sc     -out:file:o  testing_out/grouped/ZCO

In [24]:
files = sorted(glob("testing_out/grouped/*_IA*.sc"), reverse=True)
files

['testing_out/grouped/ZCON_37_N3_C1_2_IA.sc',
 'testing_out/grouped/ZCON_37_3+1__numH4__from-22.25__to17.45__grC-ABD_IA.sc',
 'testing_out/grouped/ZCON_37_3+1__numH4__from-22.25__to17.45__grC-ABD.nosegi_IA.sc',
 'testing_out/grouped/ZCON_37_3+1__numH4__from-22.25__to17.45__grAB-CD_IA.sc']

description,ZCON_37_N3_C1_2,ZCON_37_3+1__numH4__from-22.25__to17.45__grC-ABD,ZCON_37_3+1__numH4__from-22.25__to17.45__grC-ABD.nosegi,ZCON_37_3+1__numH4__from-22.25__to17.45__grAB-CD
total_score,-360.706,-309.759,-309.759,-311.503
AlaCount,2,2,2,0
NetCharge_binder,-6,-1,6,0
NetCharge_target,-3,6,-1,5
buried_minus_exposed,675.431,6191.06,188.247,3392
buried_np,2664.72,9968.53,2350.62,6242.5
buried_over_exposed,1.34,2.639,1.087,2.19
cart_bonded_monomer,0,0,0,0
cavity_volume,0,51.334,0,0
complex_normalized,-3.031,-2.868,-2.868,-2.884


In [38]:
scores_sasa = [col for col in scores.columns if "sasa" in col.lower()]
scores_sasa
with pd.option_context("display.max_rows", 1000, "display.max_colwidth", 80, "display.width", 50):
    display(scores[scores_sasa].transpose())

description,ZCON_37_N3_C1_2,ZCON_37_3+1__numH4__from-22.25__to17.45__grC-ABD,ZCON_37_3+1__numH4__from-22.25__to17.45__grC-ABD.nosegi,ZCON_37_3+1__numH4__from-22.25__to17.45__grAB-CD
contact_core_SASA,32.00000,224.000000,7.000000,124.000000
dG_cross/dSASAx100,-3.98000,-4.301000,-4.301000,-4.304000
dG_separated/dSASAx100,-3.65700,-3.735000,-3.735000,-3.534000
dSASA_hphobic,1976.47400,2153.419000,2153.419000,1895.179000
dSASA_int,2567.33300,2605.610000,2605.610000,2649.636000
dSASA_polar,590.86000,452.191000,452.191000,754.458000
degree_core_SASA,8.14300,10.012000,8.259000,9.556000
interface_buried_sasa,2567.33300,2605.610000,2605.610000,2649.636000
interface_hydrophobic_sasa,2115.63400,2184.672000,2184.672000,1981.252000
interface_polar_sasa,451.69900,420.938000,420.938000,668.384000


In [44]:
scores_hbond = [col for col in scores.columns if "loop_sc" in col.lower()]
scores_hbond
with pd.option_context("display.max_rows", 1000, "display.max_colwidth", 80, "display.width", 50):
    display(scores[scores_hbond].transpose())

description,ZCON_37_N3_C1_2,ZCON_37_3+1__numH4__from-22.25__to17.45__grC-ABD,ZCON_37_3+1__numH4__from-22.25__to17.45__grC-ABD.nosegi,ZCON_37_3+1__numH4__from-22.25__to17.45__grAB-CD
loop_sc,0.896,0.776,0.863,0.831


In [9]:
truncator.score_interface("data/ZCON_37_N3_C1_2.pdb",
                script_name="truncator/xml/04_FD_FR_and_scoring_benchmark_v3_digs_updated_schema_BUNS3_IA_select.xml",
                out_dir="testing_out/grouped", tee=False, test_run=True)

/software/rosetta/latest/bin/rosetta_scripts     -parser:protocol truncator/xml/04_FD_FR_and_scoring_benchmark_v3_digs_updated_schema_BUNS3_IA_select.xml -s data/ZCON_37_N3_C1_2.pdb     -out:file:scorefile  testing_out/grouped/ZCON_37_N3_C1_2.sc     -out:file:o  testing_out/grouped/ZCON_37_N3_C1_2.pdb     -print_pymol_selection 0     -beta     -in:file:fullatom     -renumber_pdb 1     -overwrite     -out:no_nstruct_label     -holes:dalphaball truncator/scripts/DAlphaBall.gcc     > testing_out/grouped/ZCON_37_N3_C1_2.log
Saving to: testing_out/grouped/ZCON_37_N3_C1_2_IA.sc


'testing_out/grouped/ZCON_37_N3_C1_2_IA.sc'

In [11]:

scores = truncator.read_score_files(['testing_out/grouped/ZCON_37_N3_C1_2_IA.sc'], skiprows=False, load_seq=False).set_index('description')
with pd.option_context("display.max_rows", 1000, "display.max_colwidth", 80, "display.width", 50):
    display(scores.transpose())

description,ZCON_37_N3_C1_2
total_score,-360.706
NetCharge_A,-3
NetCharge_B,-6
VirtualRootMover_root,120
cart_bonded_monomer,0
cavity_volume,0
complex_normalized,-3.031
dG_cross,-102.183
dG_cross/dSASAx100,-3.98
dG_separated,-93.888


In [ ]:
interesting_cols = """
interface_averageDegree_chA
interface_averageDegree_chB
interface_buried_sasa
interface_contactcount
interface_fxn_hydrophobic
interface_hbonds
interface_holes
interface_hydrophobic_sasa
interface_polar_sasa
interface_salt_bridges
interface_sc
interface_sc_int_area
interface_unsat_hbond1
interface_unsat_hbond2
cross_interface_energy_IA
cross_interface_hbonds_IA
delta_unsat_hbonds_IA
hbond_energy_IA
interface_delta_sasa_IA
interface_hydrophobic_sasa_IA
interface_polar_sasa_IA
exposed_hydrophobics
exposed_polars
exposed_total
""".split()
#print("\n".join(list(df.columns)))
df[interesting_cols].transpose()

In [ ]:
pd.set_option('display.max_rows', None)
pd.options.display.max_columns=None
df.transpose()

In [ ]:
import dask
from dask_jobqueue import SLURMCluster

cluster = SLURMCluster(cores=1,
                     memory="2GB",
                     queue='interactive',)
cluster.scale(4)

from dask.distributed import Client
client = Client(cluster)